In [14]:
pip install -U selenium


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\souka\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\souka\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\souka\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Scrapping

In [17]:
from datetime import datetime
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def scrape_lacentrale():
    base_url = "https://www.lacentrale.fr"
    search_url = base_url + "/listing?yearMax=2023&yearMin=2018"
    
    options = webdriver.ChromeOptions()
    options.add_argument("--headless") 
    
    driver = webdriver.Chrome(options=options)  

    car_data = []
    
    try:
        driver.get(search_url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Vehiculecard_Vehiculecard_cardBody")))
        
        page_limit = 35 

        for _ in range(page_limit):
            soup = BeautifulSoup(driver.page_source, "html.parser")
            car_listings = soup.find_all("div", class_="Vehiculecard_Vehiculecard_cardBody")
            
            for listing in car_listings:
                car_info = {}
                                
                car_year = int(listing.find("div", class_="Text_Text_text Vehiculecard_Vehiculecard_characteristicsItems Text_Text_body2").text.strip())
                current_year = datetime.now().year
                if current_year - car_year <= 5:
                    car_info["brand"] = listing.find("h2", class_="Text_Text_text Vehiculecard_Vehiculecard_title Text_Text_subtitle2").text.strip()

                    seats_element = listing.find("span", class_="Text_Text_text Vehiculecard_Vehiculecard_price Text_Text_subtitle2")
                    car_info["price"] = seats_element.text.strip() if seats_element else "N/A"

                    car_characteristics = listing.find("div", class_="Vehiculecard_Vehiculecard_characteristics")
                    car_info["year"] = int(car_characteristics.find_all("div", class_="Text_Text_text")[0].text.strip())
                    car_info["kilometers"] = int(car_characteristics.find_all("div", class_="Text_Text_text")[1].text.strip().replace(" km", "").replace(" ", ""))
                    car_info["gear_box"] = car_characteristics.find_all("div", class_="Text_Text_text")[2].text.strip()
                    car_info["energy"] = car_characteristics.find_all("div", class_="Text_Text_text")[3].text.strip()

                    car_data.append(car_info)
            
            pagination_div = driver.find_element(By.CLASS_NAME, "numberPages")
            pagination_links = pagination_div.find_elements(By.TAG_NAME, "a")
            
            next_page_link = None
            for link in pagination_links:
                if "active" in link.get_attribute("class"):
                    next_page_link = link
                    break

            if next_page_link is None:
                break
            
            next_page_url = next_page_link.get_attribute("href")
            driver.get(next_page_url)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Vehiculecard_Vehiculecard_cardBody")))
            
    finally:
        driver.quit() 

    return car_data

if __name__ == "__main__":
    scraped_data = scrape_lacentrale()
    for car in scraped_data:
        print(car)


{'brand': 'FIAT 500 X phase 2', 'price': '23 780 €', 'year': 2021, 'kilometers': 56525, 'gear_box': 'Automatique', 'energy': 'Essence'}
{'brand': 'AUDI Q3 II SPORTBACK', 'price': '61 540 €', 'year': 2020, 'kilometers': 51309, 'gear_box': 'Automatique', 'energy': 'Essence'}
{'brand': 'VOLKSWAGEN GOLF VII SW phase 2', 'price': '19 530 €', 'year': 2020, 'kilometers': 79859, 'gear_box': 'Manuelle', 'energy': 'Diesel'}
{'brand': 'PEUGEOT 3008 II phase 2', 'price': '41 420 €', 'year': 2023, 'kilometers': 15999, 'gear_box': 'Automatique', 'energy': 'Hybrides'}
{'brand': 'PEUGEOT 3008 II', 'price': '27 840 €', 'year': 2021, 'kilometers': 88634, 'gear_box': 'Automatique', 'energy': 'Diesel'}
{'brand': 'LAND ROVER RANGE ROVER SPORT II phase 2', 'price': '80 620 €', 'year': 2021, 'kilometers': 78748, 'gear_box': 'Automatique', 'energy': 'Hybrides'}
{'brand': 'AUDI A3 IV SPORTBACK', 'price': '33 050 €', 'year': 2023, 'kilometers': 15336, 'gear_box': 'Automatique', 'energy': 'Essence'}
{'brand': 'A

 Création du dataset 

In [18]:
import pandas as pd

df = pd.DataFrame(scraped_data)
df

,brand,price,year,kilometers,gear_box,energy
0,FIAT 500 X phase 2,23 780 €,2021,56525,Automatique,Essence
1,AUDI Q3 II SPORTBACK,61 540 €,2020,51309,Automatique,Essence
2,VOLKSWAGEN GOLF VII SW phase 2,19 530 €,2020,79859,Manuelle,Diesel
3,PEUGEOT 3008 II phase 2,41 420 €,2023,15999,Automatique,Hybrides
4,PEUGEOT 3008 II,27 840 €,2021,88634,Automatique,Diesel
...,...,...,...,...,...,...
590,VOLKSWAGEN TAIGO,26 190 €,2023,19422,Manuelle,Essence
591,PEUGEOT 208 II,25 570 €,2023,12,Manuelle,Essence
592,JEEP AVENGER,46 450 €,2024,3422,Automatique,Électrique
593,VOLVO XC60 II,42 290 €,2019,98580,Automatique,Hybrides


Nettoyage / Conversion des données

In [19]:
def convert_price(price_text):
    price_cleaned = price_text.replace('€', '').replace(' ', '').replace(',', '')
    return float(price_cleaned)

df['price'] = df['price'].apply(convert_price)
df

,brand,price,year,kilometers,gear_box,energy
0,FIAT 500 X phase 2,23780.0,2021,56525,Automatique,Essence
1,AUDI Q3 II SPORTBACK,61540.0,2020,51309,Automatique,Essence
2,VOLKSWAGEN GOLF VII SW phase 2,19530.0,2020,79859,Manuelle,Diesel
3,PEUGEOT 3008 II phase 2,41420.0,2023,15999,Automatique,Hybrides
4,PEUGEOT 3008 II,27840.0,2021,88634,Automatique,Diesel
...,...,...,...,...,...,...
590,VOLKSWAGEN TAIGO,26190.0,2023,19422,Manuelle,Essence
591,PEUGEOT 208 II,25570.0,2023,12,Manuelle,Essence
592,JEEP AVENGER,46450.0,2024,3422,Automatique,Électrique
593,VOLVO XC60 II,42290.0,2019,98580,Automatique,Hybrides


In [20]:
missing_count = df.isna().sum()
missing_count

brand         0
price         0
year          0
kilometers    0
gear_box      0
energy        0
dtype: int64

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595 entries, 0 to 594
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   brand       595 non-null    object 
 1   price       595 non-null    float64
 2   year        595 non-null    int64  
 3   kilometers  595 non-null    int64  
 4   gear_box    595 non-null    object 
 5   energy      595 non-null    object 
dtypes: float64(1), int64(2), object(3)
memory usage: 28.0+ KB


Chargement des données en csv

In [22]:
# Division des données en train_data et test_data
train_data = df.sample(frac=0.8, random_state=42)  # Par exemple, utilisez 80% des données pour l'entraînement
test_data = df.drop(train_data.index)

# Sauvegardez les données d'entraînement en CSV
train_data.to_csv("train_data.csv", index=False)

# Sauvegardez les données de test en CSV
test_data.to_csv("test_data.csv", index=False)


Modèle de regression linéaire

In [23]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\souka\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [24]:
pip install matplotlib


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\souka\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Création d'une copie du DataFrame original
encoded_df = df.copy()

# Encodage des colonnes catégorielles
label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False, drop='first')

encoded_df['brand'] = label_encoder.fit_transform(encoded_df['brand'])
encoded_df['gear_box'] = label_encoder.fit_transform(encoded_df['gear_box'])
encoded_df['energy'] = label_encoder.fit_transform(encoded_df['energy'])

# Conversion de l'année en entier
encoded_df['year'] = encoded_df['year'].astype(int)

# Sélection des caractéristiques et de la variable cible
features = encoded_df[[ 'kilometers', 'year', 'gear_box', 'energy']]
target = encoded_df['price']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Création et entraînement du modèle de régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Coefficient de détermination (R²)
r2 = r2_score(y_test, predictions)
print(f"Coefficient de détermination (R²): {r2:.2f}")



Coefficient de détermination (R²): 0.39


In [26]:
df

,brand,price,year,kilometers,gear_box,energy
0,FIAT 500 X phase 2,23780.0,2021,56525,Automatique,Essence
1,AUDI Q3 II SPORTBACK,61540.0,2020,51309,Automatique,Essence
2,VOLKSWAGEN GOLF VII SW phase 2,19530.0,2020,79859,Manuelle,Diesel
3,PEUGEOT 3008 II phase 2,41420.0,2023,15999,Automatique,Hybrides
4,PEUGEOT 3008 II,27840.0,2021,88634,Automatique,Diesel
...,...,...,...,...,...,...
590,VOLKSWAGEN TAIGO,26190.0,2023,19422,Manuelle,Essence
591,PEUGEOT 208 II,25570.0,2023,12,Manuelle,Essence
592,JEEP AVENGER,46450.0,2024,3422,Automatique,Électrique
593,VOLVO XC60 II,42290.0,2019,98580,Automatique,Hybrides


xgboost

In [27]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Users\souka\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [28]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Créer une copie du DataFrame pour les modifications
df_modified = df.copy()

# Prétraitement
label_encoder = LabelEncoder()

df_modified['brand'] = label_encoder.fit_transform(df_modified['brand'])
df_modified['gear_box'] = label_encoder.fit_transform(df_modified['gear_box'])
df_modified['energy'] = label_encoder.fit_transform(df_modified['energy'])
df_modified['year'] = df_modified['year'].astype(int)


# Sélection des caractéristiques et de la variable cible
features = df_modified[['brand', 'kilometers', 'year', 'gear_box', 'energy']]  # Utilisation de toutes les caractéristiques
target = df_modified['price']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Créer et entraîner le modèle XGBoost
model = xgb.XGBRegressor()
model.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
predictions = model.predict(X_test)


# Calcul de l'erreur quadratique moyenne (RMSE)
r2 = r2_score(y_test, predictions)
print(f"Coefficient de détermination (R²): {r2:.2f}")
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"Erreur quadratique moyenne (RMSE): {rmse:.2f}")


Coefficient de détermination (R²): 1.00
Erreur quadratique moyenne (RMSE): 0.00


Random Forest

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


label_encoder = LabelEncoder()
onehot_encoder = OneHotEncoder(sparse=False, drop='first')

df['brand'] = label_encoder.fit_transform(df['brand'])
df['gear_box'] = label_encoder.fit_transform(df['gear_box'])
df['energy'] = label_encoder.fit_transform(df['energy'])
df['year'] = df['year'].astype(int)

features = df[['brand', 'kilometers', 'year', 'gear_box', 'energy']]
target = df['price']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Créeation et entraînement du modèle Random Forest
model = RandomForestRegressor()
model.fit(X_train, y_train)

# prédictions sur l'ensemble de test
predictions = model.predict(X_test)

# Calculer le coefficient de détermination (R²)
r2 = r2_score(y_test, predictions)
print(f"Coefficient de détermination (R²): {r2:.2f}")
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"Erreur quadratique moyenne (RMSE): {rmse:.2f}")



Coefficient de détermination (R²): 1.00
Erreur quadratique moyenne (RMSE): 0.00
